In [10]:
import os
os.chdir('..')
from inputGetter import get_input
import math

content = get_input(2024, 15)[:-1]

In [11]:
content = content.split('\n')

line_id = 0
mp = []

start_r, start_c = -1, -1

while len(content[line_id]) > 0:
    mp.append(list(content[line_id]))
    
    if start_r == -1:
        for col_id, cell in enumerate(content[line_id]):
            if cell == '@':
                start_r, start_c = line_id, col_id
    
    line_id += 1

instructions = ''
while line_id < len(content):
    instructions += content[line_id]
    line_id += 1

directions = {'^': (-1, 0), '>': (0, 1), 'v': (1, 0), '<': (0, -1)}

for inst in instructions:
    add_r, add_c = directions[inst]
    new_r, new_c = start_r + add_r, start_c + add_c
    
    if mp[new_r][new_c] == '.':
        mp[start_r][start_c] = '.'
        mp[new_r][new_c] = '@'
        start_r, start_c = new_r, new_c
    elif mp[new_r][new_c] == 'O':
        cpy_r, cpy_c = new_r, new_c
        while mp[new_r][new_c] != '#' and mp[new_r][new_c] != '.':
            new_r, new_c = new_r + add_r, new_c + add_c
        
        if mp[new_r][new_c] == '.':
            mp[new_r][new_c] = 'O'
            mp[cpy_r][cpy_c] = '@'
            mp[start_r][start_c] = '.'
            start_r, start_c = cpy_r, cpy_c

res_a = 0
for r in range(len(mp)):
    for c in range(len(mp[0])):
        if mp[r][c] == 'O':
            res_a += r * 100 + c

print(f'Part A: {res_a}')

Part A: 2453406


In [12]:
# Transform map for part B
line_id = 0
mp = []
start_r, start_c = -1, -1
while len(content[line_id]) > 0:
    new_ls = []
    for c in content[line_id]:
        if c == '#':
            new_ls.append('#')
            new_ls.append('#')
        elif c == '.':
            new_ls.append('.')
            new_ls.append('.')
        elif c == 'O':
            new_ls.append('[')
            new_ls.append(']')
        else:
            start_r, start_c = line_id, len(new_ls)
            new_ls.append('@')
            new_ls.append('.')
    mp.append(new_ls)
    line_id += 1

sz_r, sz_c = len(mp), len(mp[0])

for inst in instructions:
    add_r, add_c = directions[inst]
    new_r, new_c = start_r + add_r, start_c + add_c
    
    if mp[new_r][new_c] == '.':
        mp[start_r][start_c] = '.'
        mp[new_r][new_c] = '@'
        start_r, start_c = new_r, new_c
        
    # Main issue is with this one, because you can move multiple boxes
    elif mp[new_r][new_c] == '[' or mp[new_r][new_c] == ']':
        if inst == '^' or inst == 'v':
            valid_move = True
            
            affected_blocks = [set()]
            affected_blocks[-1].add((start_c, start_c))
            
            while 0 <= new_r < sz_r and valid_move:
                curr_lv = set()
                for block in affected_blocks[-1]:
                    opening, closing = block
                    
                    if mp[new_r][opening] == '#' or mp[new_r][closing] == '#':
                        valid_move = False
                        break
                    
                    if mp[new_r][opening] == '[':
                        curr_lv.add((opening, opening+1))
                    if mp[new_r][opening] == ']':
                        curr_lv.add((opening-1, opening))
                    if mp[new_r][closing] == '[':
                        curr_lv.add((closing, closing+1))
                    if mp[new_r][closing] == ']':
                        curr_lv.add((closing-1, closing))
                
                if len(curr_lv) == 0:
                    break
                
                affected_blocks.append(curr_lv)
                new_r += add_r
            
            if valid_move:
                for i in range(len(affected_blocks)-1, -1, -1):
                    for block in affected_blocks[i]:
                        if block[0] != block[1]:
                            mp[new_r][block[0]] = '['
                            mp[new_r][block[1]] = ']'
                        else:
                            mp[new_r][block[0]] = '@'
                            start_r = new_r
                        mp[new_r - add_r][block[0]] = '.'
                        mp[new_r - add_r][block[1]] = '.'
                    new_r -= add_r
        else:
            while mp[new_r][new_c] != '.' and mp[new_r][new_c] != '#':
                new_c += add_c
            
            if mp[new_r][new_c] == '.':
                while new_c != start_c:
                    mp[new_r][new_c] = mp[new_r][new_c - add_c]
                    new_c -= add_c
        
                mp[start_r][start_c] = '.'
                start_c += add_c
                mp[start_r][start_c] = '@'

res_b = 0
for r in range(len(mp)):
    for c in range(len(mp[0])):
        if mp[r][c] == '[':
            res_b += r * 100 + c

print(f'Part B: {res_b}')

Part B: 1521952
